In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
from typing import List, Tuple, Dict

In [3]:
myint = tf.int32
myfloat = tf.float32

In [4]:
x_shape = np.array([32, 32, 3])
y_shape = np.array([10])
1st_convolution_shape = np.array([5, 5, 3, 32]) # with pad 2 stride 1
1st_layer_shape = np.array([32, 32, 32])
2nd_pool_shape = np.array([2, 2]) # with stride 2
2nd_layer_shape = np.array([16, 16, 32])
# 3rd normalization
4th_convolution_shape = np.array([5, 5, 32, 64]) # with pad 2 stride 1
4th_layer_shape = np.array([16, 16, 64])
5th_pool_shape = np.array([2, 2])  # with stride 2
5th_layer_shape = np.array([8, 8, 64])
# 6th normalization
6th_dence_shape = np.array([4096, 1024])  # 8*8*64
6th_layer_shape = np.array([1024])
7th_weight_shape = np.array([1024, 10])
7th_layer_shape = np.array([10])  # output

SyntaxError: invalid syntax (<ipython-input-4-e9f52c98e358>, line 3)

In [ ]:
class DataLoader:
    
    def __init__(self):
        self.data, self.labels = self._load_data()
        
    def _load_data(self):
        path = 'data/cifar-10-batches-py/'
        file_format = 'data_batch_{}'
        train_data = []
        train_labels = []
        with open(path + file_format.format(1), 'r') as f:
            for i in range(5):
                f = open(path + file_format.format(i+1), 'rb')
                data = pickle.load(f, encoding='bytes')
                data, labels = self._prepare_train_data(data)
                train_data.append(self._get_reshaped_data(data))
                train_labels.append(labels)
        return train_data, train_labels
    
    def _prepare_train_data(self, dict_data):
        data = dict_data[b'data']
        labels = dict_data[b'labels']
        return data, labels
    
    def _get_reshaped_data(self, data):
        # 元データのshape check
        original_data_shape = [10000, 32*32*3]
        assert_shape(data, original_data_shape)

        reshaped_data_shape = [10000, 32, 32, 3]
        reshaped_data = np.reshape(data, [10000, 3, 32, 32])
        reshaped_data = np.transpose(reshaped_data, [0, 2, 3, 1])

        # 修正したデータのshape check
        assert_shape(reshaped_data, reshaped_data_shape)
        return reshaped_data

In [ ]:
datasource = DataLoader()

In [ ]:
def assert_shape(target: np.ndarray, shape):
    if isinstance(shape, list):
        shape = np.array(shape)
    
    assert isinstance(target, np.ndarray), "target is not ndarray"
    assert isinstance(shape, np.ndarray), "shape is not ndarray"
    
    target_shape = target.shape
    assert len(target_shape) == len(shape), 'dimension is not equal'
    for (i, s) in enumerate(shape):
        assert target_shape[i] == s, "size of target[{}] is not {}".format(i, s)

In [ ]:
print(datasource.data[0].shape)
print(len(datasource.labels[0]))

# network

In [ ]:
x = tf.placeholder(dtype=myfloat, shape=x_shape, name='x')
y = tf.placeholder(dtype=myint, shape=y_shape, name='y')